## Converting Wikidata Entity Schema E130: CSV into ShEx

Links
* https://github.com/dcmi/dcap/blob/master/prototypes/wikidata_painting/E130.ipynb - this notebook
* https://www.wikidata.org/wiki/EntitySchema:E130 - Wikidata Entity Schema E130
  * cached [in ShExC](https://github.com/dcmi/dcap/blob/master/prototypes/wikidata_painting/E130.shexc)
  * cached [in ShExJ](https://github.com/dcmi/dcap/blob/master/prototypes/wikidata_painting/E130.shexj)
* https://github.com/dcmi/dcap/blob/master/prototypes/wikidata_painting/E130.csv - CSV expression equivalent to E130

This notebook documents a proof-of-concept script that takes a CSV expression of Wikidata Entity Schema E130 (a ShEx schema in ShExC syntax) and converts it from CSV back into ShExC syntax.

In [1]:
import csv
reader = csv.DictReader(open('E130.csv', newline='', encoding='utf-8-sig'))
reader.fieldnames

['Entity_name',
 'Entity_label',
 'Property',
 'Property_label',
 'Cardinality',
 'Value',
 'Value_type',
 'Annotation']

In [2]:
all_rows = [dict(row) for row in reader]
all_rows

[{'Entity_name': 'painting',
  'Entity_label': 'Painting',
  'Property': 'wdt:P31',
  'Property_label': 'Is a',
  'Cardinality': '1..1',
  'Value': 'wd:Q3305213',
  'Value_type': 'URI',
  'Annotation': 'instance of "painting"'},
 {'Entity_name': '',
  'Entity_label': '',
  'Property': 'wdt:P571',
  'Property_label': 'Date of inception',
  'Cardinality': '0..1',
  'Value': '',
  'Value_type': 'DateTime',
  'Annotation': ''},
 {'Entity_name': '',
  'Entity_label': '',
  'Property': 'wdt:P276',
  'Property_label': 'Location',
  'Cardinality': '1..-1',
  'Value': '',
  'Value_type': '',
  'Annotation': ''},
 {'Entity_name': '',
  'Entity_label': '',
  'Property': 'wdt:P1476',
  'Property_label': 'Title',
  'Cardinality': '1..-1',
  'Value': '',
  'Value_type': '',
  'Annotation': ''},
 {'Entity_name': '',
  'Entity_label': '',
  'Property': 'wdt:P195',
  'Property_label': 'Collection',
  'Cardinality': '1..-1',
  'Value': '',
  'Value_type': '',
  'Annotation': ''},
 {'Entity_name': '',
  

In [3]:
prefixes = """PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>"""

In [4]:
end_matter = """}

# Proposed test for SPARQL - Top 25 paintings
# SELECT ?id WHERE { VALUES ?id { wd:Q12418 wd:Q45585 wd:Q175036 wd:Q29530 wd:Q185372 wd:Q219831 wd:Q151047 wd:Q208758 wd:Q25729 wd:Q154469 wd:Q474338 wd:Q328523 wd:Q321303 wd:Q1892 745 wd:Q334138 wd:Q1091086 wd:Q698487 wd:Q212616 wd:Q152509 wd:Q152867 wd:Q220859 wd:Q734834  } }"""

In [5]:
schema = []
start = ''
started = False
for row in all_rows:
    entity_name = row['Entity_name']
    property = row['Property']
    property_label = row['Property_label']
    cardinality = row['Cardinality']
    value = row['Value']           # Node Constraint
    value_type = row['Value_type'] # Node Constraint
    annotation = row['Annotation']
    vtype = ''
    card = ''
    if not start:
        if entity_name:
            start_node = entity_name
            start = f"""start = @<{start_node}>"""
            schema.append(f"{start}\n\n")
    if entity_name:
        if started:
            schema.append("}\n\n")
        shape_name = f"<{entity_name}>" + " {\n"
        schema.append(shape_name)
        started = True
    if property_label:
        schema.append(f"    # {repr(property_label)} {annotation}\n")
    if cardinality == "1..1":
        card = ''
    elif cardinality == "0..1":
        card = '?'
    elif cardinality == "1..-1":
        card = '+'
    if value_type == "DateTime":
        vtype = " xsd:dateTime"
        schema.append(f"    {property}{vtype}{card} ;\n")
    elif value_type == "URIStem":
        value = value + "~"
        schema.append(f"    {property} [{value}] {card} ;\n")
    elif value_type == "Entity_name":
        value = f"@<{value}>"
        schema.append(f"    {property} {value}{card} ;\n")
    elif not value:
        schema.append(f"    {property} . {card} ;\n")
    elif value:
        schema.append(f"    {property} [{value}] {card} ;\n")

print(prefixes, "\n")
for line in schema:
    print(line, end="")
print("\n", end_matter)

PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#> 

start = @<painting>

<painting> {
    # 'Is a' instance of "painting"
    wdt:P31 [wd:Q3305213]  ;
    # 'Date of inception' 
    wdt:P571 xsd:dateTime? ;
    # 'Location' 
    wdt:P276 . + ;
    # 'Title' 
    wdt:P1476 . + ;
    # 'Collection' 
    wdt:P195 . + ;
    # 'Creator' 
    wdt:P170 @<creator>+ ;
}

<creator> {
    # 'Is a' instance of "creator"
    wdt:P31 [wd:~]  ;

 }

# Proposed test for SPARQL - Top 25 paintings
# SELECT ?id WHERE { VALUES ?id { wd:Q12418 wd:Q45585 wd:Q175036 wd:Q29530 wd:Q185372 wd:Q219831 wd:Q151047 wd:Q208758 wd:Q25729 wd:Q154469 wd:Q474338 wd:Q328523 wd:Q321303 wd:Q1892 745 wd:Q334138 wd:Q1091086 wd:Q698487 wd:Q212616 wd:Q152509 wd:Q152867 wd:Q220859 wd:Q734834  } }


### Result

The resulting schema (above) is identical with [Wikidata Entity Schema E130](https://www.wikidata.org/wiki/EntitySchema:E130).

This can be tested by cut-and-pasting the schemas into [ShEx2 online parser](https://rawgit.com/shexSpec/shex.js/master/packages/shex-webapp/doc/shex-simple.html), leaving the data box blank, and running "validate".
* E130 (ShExC) converted into ShExJ: [the result](https://github.com/dcmi/dcap/blob/master/painting/E130.shexj)
* The ShExC schema generated above, converted into ShExJ: [the result](https://github.com/dcmi/dcap/blob/master/painting/E130.shexj_generated)

In [6]:
import pandas as pd
pd.read_csv('E130.csv')

,Entity_name,Entity_label,Property,Property_label,Cardinality,Value,Value_type,Annotation
0,painting,Painting,wdt:P31,Is a,1..1,wd:Q3305213,URI,"instance of ""painting"""
1,NaN,NaN,wdt:P571,Date of inception,0..1,NaN,DateTime,NaN
2,NaN,NaN,wdt:P276,Location,1..-1,NaN,NaN,NaN
3,NaN,NaN,wdt:P1476,Title,1..-1,NaN,NaN,NaN
4,NaN,NaN,wdt:P195,Collection,1..-1,NaN,NaN,NaN
5,NaN,NaN,wdt:P170,Creator,1..-1,creator,Entity_name,NaN
6,creator,Artist,wdt:P31,Is a,1..1,wd:,URIStem,"instance of ""creator"""
